In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
#import nltk
#from nltk.corpus import stopwords
#nltk.download('stopwords')
#import spacy
#import gensim
#from gensim.parsing.preprocessing import remove_stopwords
from nlppreprocess import NLP
#import imbalanced-learn
#from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
#from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn import metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import plot_confusion_matrix

In [2]:
#DATASET_ENCODING = "ISO-8859-1"
#if columns are not correctly identified in excel u can place ur own
#DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
import matplotlib.style as style
style.use('ggplot')

In [3]:
#loading in dataset
df = pd.read_csv("C:/Users/Omer/Documents/datasetsent/fakejobprediction/fake_job_postings.csv")


In [4]:
#checking first 5 entries in the data
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [5]:
#Fillin NaN values with blanks or could fill with 0s
#df.fillna(' ',inplace=True)
#Just for the graphs so they do not appear blank after removing NaNs
df.fillna('NA',inplace=True)

In [6]:
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NA,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NA,0,1,0,Other,Internship,NA,NA,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NA,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NA,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NA,NA,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NA,0,1,0,NA,NA,NA,NA,NA,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NA,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NA,NA,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [ ]:
#Imbalanced dataset there are less fraudulent job postings
sns.countplot(x='fraudulent', data=df).set_title("Imbalanced job postings")
#plt.show()
#plt.savefig('C:/Users/Omer/Documents/github-repos/Fake job postings/Results/Graphs/imbalanced-datasett.png',dpi=600)

In [ ]:
#grouping all fraudu
df.groupby('fraudulent').count()['title'].reset_index().sort_values(by='title',ascending=False)

In [ ]:
print(df.columns)

In [ ]:
#df.describe()
sns.set_style('darkgrid')
plt.figure()
sns.countplot(hue=df.fraudulent,x=df.employment_type)
plt.title('Fraudulence distribution based on Employment Type')
plt.xlabel('Employment Type')
plt.ylabel('No. of Jobs')
#plt.show()
#plt.savefig('C:/Users/Omer/Documents/github-repos/Fake job postings/Results/Graphs/fraud-based-on-employment-type.png',dpi=600)

In [ ]:
#some of the industries have unusually large names so removing extra words
#def indus(industry):
#    r = industry.split('and')
#    return r[0]

#df['industry'] = df.industry.apply(indus)

In [ ]:
#indusvlist = np.array(list(industry.values())).astype(float)
#indusvlist = list(industry.values())
#indusvlist
#indusklist = np.array(list(industry.keys()))
#indusklist = list(industry.keys())
#indusklist

In [ ]:
#fraud based on industry type
#sns.set_style('darkgrid')
#sns.set_context('poster') 
#industry = dict(df.industry.value_counts()[:11])
#del industry[' ']
#indusvlist = np.array(list(industry.values())).astype(float)
plt.figure(figsize=(35,40))
#sns.set(font_scale=1.2,palette='deep', color_codes=False,context='notebook', style='darkgrid', font='sans-serif', font_scale=1, rc=None)
sns.set(context='notebook', style='darkgrid', palette=None, font='sans-serif', font_scale=1.2, color_codes=False, rc=None)
#seaborn cannot handle dict values so an alternative is to pass them as a list.
sns.countplot(y='industry', hue='fraudulent',data=df )#,palette="RdBu_r")
#sns.barplot
#sns.countplot(x=df.industry,hue=df.fraudulent)
#g._legend.remove()
#plt.bar(df.fraudulent,df.industry)
#plt.bar(df.fraudulent)
#barwidth =0.25
#plt.bar(indusklist,indusvlist)
#ax.bar(X + 0.50, df.fraudulent, color = 'r', width = 0.25)
#plt.rc('axes', titlesize=50)
plt.title('Jobs posted based on industry',fontsize=22)
#plt.xticks(('Information Technology','Computer Software','Internet','Marketing','Education Management','Financial Services','Hospital & Health Care','Consumer Services','Telecommunications','Oil & Energy'))
plt.ylabel('Industry Type',fontsize=18)
plt.xlabel('No. of Jobs',fontsize=18)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.tight_layout()
#plt.show()
figure = plt.gcf() # get current figure
figure.set_size_inches(35,40)
plt.savefig('C:/Users/Omer/Documents/github-repos/Fake job postings/Results/Graphs/fraud-based-on-industry-type.png',dpi=600)

In [ ]:
#fraud = dict(df.fraudulent.value_counts()[:11])
#del fraud[' ']
#fraudli = list(fraud.values())
#fraudli

In [ ]:
#jobs posted based on function / fraud based on function
plt.figure(figsize=(20,30))
#import matplotlib.style as style
style.use('ggplot')
sns.set(context='notebook', style='darkgrid', palette=None, font='sans-serif', font_scale=1.2, color_codes=False, rc=None)
sns.countplot(y='function', hue='fraudulent',data=df)
plt.title('Jobs posted based on function',size=22)
plt.ylabel('Function Type', fontsize=18)
plt.xlabel('No. of Jobs',fontsize=18)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.tight_layout()
#plt.show()
plt.savefig('C:/Users/Omer/Documents/github-repos/Fake job postings/Results/Graphs/fraud-based-on-function-type.png',dpi=600)

In [ ]:
#required exp vs fraudulent
plt.figure(figsize=(20,20))
#import matplotlib.style as style
style.use('ggplot')
sns.set(context='notebook', style='darkgrid', palette=None, font='sans-serif', font_scale=1.2, color_codes=False, rc=None)
sns.countplot(x='required_experience', hue='fraudulent',data=df)
plt.title('Jobs posted based on required experience', size=22)
plt.ylabel('No. of Jobs', fontsize=18)
plt.xlabel('Experience Type',fontsize=18)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.tight_layout()
#plt.show()
plt.savefig('C:/Users/Omer/Documents/github-repos/Fake job postings/Results/Graphs/fraud-based-on-required-experience-type.png',dpi=600)

In [ ]:
#since the country column contains exact locations, this function will allow us to get only the country from the column by splitting the country when it runs into a ,
def countryonly(location):
    l = location.split(',')
    return l[0]

df['country'] = df.location.apply(countryonly)

In [ ]:
#postings per country
country = dict(df.country.value_counts()[:11])
#del country[' ']
plt.figure(figsize=(8,6))
plt.title('Country wise job postings', size=18)
plt.bar(country.keys(), country.values())
plt.ylabel('No. of jobs', size=10)
plt.xlabel('Countries', size=10)
#plt.savefig('C:/Users/Omer/Documents/github-repos/Fake job postings/Results/Graphs/country-wise-jobs.png',dpi=600)

In [10]:
#plt.figure(figsize=(35,40))
#sns.countplot(y='location', hue='fraudulent',data=df )
#plt.title('Jobs posted based on country',fontsize=22)
#plt.xticks(('Information Technology','Computer Software','Internet','Marketing','Education Management','Financial Services','Hospital & Health Care','Consumer Services','Telecommunications','Oil & Energy'))
#plt.ylabel('location',fontsize=18)
#plt.xlabel('No. of Jobs',fontsize=18)
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
#plt.tight_layout()
#plt.show()

In [ ]:
#required educations vs fraudulent
plt.figure(figsize=(20,20))
sns.countplot(y='required_education', hue='fraudulent',data=df)
plt.title('Jobs posted based on required education',size=22)
plt.ylabel('Required Education', fontsize=18)
plt.xlabel('No. of Jobs',fontsize=18)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.tight_layout()
#plt.show()
plt.savefig('C:/Users/Omer/Documents/github-repos/Fake job postings/Results/Graphs/fraud-based-on-required-education-type.png',dpi=600)

In [ ]:
#plt.figure(figsize=(20,20))
#sns.countplot(x='telecommuting', hue='fraudulent',data=df)
#plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
#plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
#plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
#plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
#plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
#plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
#plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
#plt.title('Jobs posted based on required education')
#plt.ylabel('Required Education', fontsize=MEDIUM_SIZE)
#plt.xlabel('No. of Jobs',fontsize=MEDIUM_SIZE)
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
#plt.tight_layout()
#plt.show()

In [ ]:
#combining texts into a single column for processing
df['text']=df['title']+' '+df['location']+' '+df['company_profile']+' '+df['description']+' '+df['requirements']+' '+df['benefits']
del df['title']
del df['location']
del df['department']
del df['company_profile']
del df['description']
del df['requirements']
del df['benefits']
del df['required_experience']
del df['required_education']
del df['industry']
del df['function']
#del df['country']
#use this to delete columns
#df.drop('ids', axis=1, inplace=True)
#df.drop('date', axis=1, inplace=True)
#df.drop('flag', axis=1, inplace=True)
#df.drop('user', axis=1, inplace=True)

In [ ]:
print(df.columns)

In [ ]:
#getting rid of the columns of no use
columns=['job_id', 'telecommuting', 'has_company_logo', 'has_questions', 'salary_range', 'employment_type']
for col in columns:
    del df[col]


In [ ]:
df.head()

In [ ]:
#checking texts before preproc
df['fraudulent'][98]

In [ ]:
nlp = NLP()

In [ ]:
#Since its an imbalanced set, I am undersampling the realjobpostings to take only 866 jobs to match the fake ones and preprocessing for stopwords.
undersampledrealjobs = []
sentences = list(df[df.fraudulent==0].text[:866])
#len(sentences)
#sentences[866]
for sen in sentences:
    undersampledrealjobs.append(nlp.process(sen))

In [ ]:
#undersampledrealjobs[98]
len(undersampledrealjobs)

In [ ]:
#Storying all 866 fraudulent text in list and preprocessing it.
fakejobs = []
fakesentences = list(df[df.fraudulent==1].text)
#len(sentences)
#sentences[866]
for sent in fakesentences:
    fakejobs.append(nlp.process(sent))

In [ ]:
len(fakejobs)

In [ ]:
#zeroes for real jobs 
realjoblabel = np.zeros(866)
#np.array(realjoblabel)
#ones for fake jobs
fakejoblabel = np.ones(866)
#fakejoblabel

In [ ]:
#sentences[0]
#realjoblabel

In [ ]:
#Appending fakejobs into realjobs
for x in fakejobs:
    undersampledrealjobs.append(x)

In [ ]:
len(undersampledrealjobs)

In [ ]:
#concatenating fakejoblabels in real
joblab = np.concatenate((realjoblabel,fakejoblabel),axis=None)
len(joblab)

In [ ]:
#X_under, y_under = undersample.fit_resample(fakejobposarr,fraudarr)
#fraudliarr=np.array(fraudli)
#fakejobposarr=np.array(fakejobpos)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(undersampledrealjobs,joblab, test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
tv_train_reviews=tv.fit_transform(X_train)
#transformed test reviews
tv_test_reviews=tv.transform(X_test)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

In [ ]:
#Multinomial naive bayes
mnb=MultinomialNB()

In [ ]:
mnb.fit(tv_train_reviews,y_train)

In [ ]:
mnbpred = mnb.predict(tv_test_reviews)

In [ ]:
classmnb = classification_report(y_test,mnbpred)
print(classmnb)

In [ ]:
print('F1score: ', metrics.f1_score(y_test, mnbpred, average='macro'))
#print('Precision: ', metrics.precision_score(y_test, mnbpred, average='macro'))
#print('Recall: ', metrics.recall_score(y_test, mnbpred, average='macro'))

In [ ]:
plot_confusion_matrix(mnb, tv_test_reviews, y_test, cmap='Blues')
plt.grid(None)

In [ ]:
#Random forest classifier
rf=RandomForestClassifier(criterion="entropy", max_features="auto")

In [ ]:
rf.fit(tv_train_reviews,y_train)

In [ ]:
rfpred=rf.predict(tv_test_reviews)

In [ ]:
classrf = classification_report(y_test,rfpred)
print(classrf)

In [ ]:
plot_confusion_matrix(rf, tv_test_reviews, y_test, cmap='Blues')
plt.grid(None)

In [ ]:
print('F1score: ', metrics.f1_score(y_test, rfpred, average='macro'))
#print('Precision: ', metrics.precision_score(y_test, rfpred, average='macro'))
#print('Recall: ', metrics.recall_score(y_test, rfpred, average='macro'))

In [ ]:
#Logistic regression
lr= LogisticRegression(solver='liblinear')

In [ ]:
lr.fit(tv_train_reviews,y_train)

In [ ]:
lrpred=lr.predict(tv_test_reviews)

In [ ]:
classlr = classification_report(y_test,lrpred)
print(classlr)

In [ ]:
plot_confusion_matrix(lr, tv_test_reviews, y_test, cmap='Blues')
plt.grid(None)

In [ ]:
#SVM
sv = svm.SVC(kernel='rbf')

In [ ]:
sv.fit(tv_train_reviews,y_train)

In [ ]:
svmmpred = sv.predict(tv_test_reviews)

In [ ]:
classsvmm = classification_report(y_test,svmmpred)
print(classsvmm)

In [ ]:
plot_confusion_matrix(sv, tv_test_reviews, y_test, cmap='Blues')
plt.grid(None)

In [ ]:
tv_train_reviews.shape